In [ ]:
import sys, serial, argparse, time
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import os
#import matplotlib.animation as animation
import scipy
from scipy import signal
from collections import deque

In [ ]:
ser = serial.Serial(
    port='com3',
    baudrate=19200,
)

In [ ]:
del ser

# Real-time signal

### Initialization - Zero setting

#### sampling rate 변함 = 값이 커지거나 급변할 때 (터치, proximity sensing) 줄어듦 (9600 아닌 값들)
#### 9600 에서는 38, 39 유지함 => 38.5 Hz

In [ ]:
sensor1, sensor2, sensor3, sensor4, index  = [], [], [], [], []
init_time = 6  #sec
window_size, Sampling_freq = 19, 53

init_period = Sampling_freq * init_time
Threshold = 16.5
hand_motion_period = 53

front_period, back_period = 50, 70

In [ ]:
def start_sensing_threshold(signal_lst, threshold, idx):
    outcome = False
    
    if signal_lst[idx - 1] > threshold:  #현재 데이터
        if signal_lst[idx - 2] < threshold:   #바로 전 데이터
            outcome = True
        else:
            pass

    return outcome
        
def end_sensing_threshold(signal_lst, threshold, idx):
    outcome = False
    
    if signal_lst[idx - 1] < threshold:  #현재 데이터
        if signal_lst[idx - 2] > threshold:   #바로 전 데이터
            outcome = True
        else:
            pass

    return outcome

In [ ]:
init_s1, init_s2, init_s3, init_s4, idx = [], [], [], [], 0
print("First Initialization")

for i in range(1,5):
    globals()['Test_threshold{}'.format(i)] = []

for i in range(100): #처음 100개 데이터는 거름
    tmp = ser.readline()

for i in range(int(init_period)):

    tmp = ser.readline()
    s1, s2, s3, s4 = tmp.decode()[:5], tmp.decode()[6:11], tmp.decode()[12:17], tmp.decode()[18:]

    try:
        s1, s2, s3, s4 = (int(s1)-10000), (int(s2)-10000), (int(s3)-10000), (int(s4)-10000)
    
    except:
        print(i, "  Error")
        pass
    
    if type(s1) == int and type(s2) == int and type(s3) == int and type(s4) == int: 
        index.append(idx)
        idx = idx + 1
        
        for j in range(1,5):
            globals()['Test_threshold{}'.format(j)].append(0)
            globals()['init_s{}'.format(j)].append(globals()['s{}'.format(j)])
            globals()['sensor{}'.format(j)].append(globals()['s{}'.format(j)])

offset_s1, offset_s2, offset_s3, offset_s4 = np.mean(init_s1), np.mean(init_s2), np.mean(init_s3), np.mean(init_s4)

init_s1, init_s2, init_s3, init_s4 = [], [], [], []

In [ ]:
for i in range(1,5):
    globals()['sensor{}_frame'.format(i)] = []  
    globals()['S{}_Threshold'.format(i)], globals()['s{}_start_frame'.format(i)], globals()['s{}_end_frame'.format(i)] = Threshold, 0, 0
    globals()['s{}_cnt'.format(i)], globals()['s{}_safe_cnt'.format(i)], globals()['s{}_safe_cnt_2'.format(i)] = 0, 0, 0
    
Final_frame, safe_cnt3, start_frame, end_frame, next_frame_period = [], 0, 0, 0, 0

while True:
    if ser.readable():
        res = ser.readline()
        s1, s2, s3, s4 = res.decode()[:5], res.decode()[6:11], res.decode()[12:17], res.decode()[18:]
                        
        try:
            s1, s2, s3, s4 = (int(s1)-10000), (int(s2)-10000), (int(s3)-10000), (int(s4)-10000)

        except:
            print(idx, " index has Error")
            pass
        
        if type(s1) == int and type(s2) == int and type(s3) == int and type(s4) == int: 
            index.append(idx)
            
            for i in range (1,5): 
                globals()['Test_threshold{}'.format(i)].append(globals()['S{}_Threshold'.format(i)])
                
                tmp2 = globals()['s{}'.format(i)] - globals()['offset_s{}'.format(i)]
                tmp2 = np.abs(np.round(tmp2,3))
                globals()['sensor{}'.format(i)].append(tmp2)
                
                if globals()['s{}_start_frame'.format(i)] == 0 and globals()['s{}_end_frame'.format(i)] == 0:  #안정해졌을 때 offset 계산해야 함
                    globals()['init_s{}'.format(i)].append(globals()['s{}'.format(i)])
                #-----------------------------------------------------------------------------------------------------------------------------------------
                
                if start_sensing_threshold(globals()['sensor{}'.format(i)], globals()['S{}_Threshold'.format(i)], idx) == True:  # Hand motion detection 시작
                    globals()['s{}_start_frame'.format(i)] = idx - 70 #period 설정, 약 20 = 0.4 초 정도 앞으로
                    globals()['s{}_safe_cnt_2'.format(i)] = globals()['s{}_safe_cnt_2'.format(i)] + 1
                    print(i, " 센서 넘음", idx)
                
                elif globals()['sensor{}'.format(i)][idx] > globals()['S{}_Threshold'.format(i)]: # Hand motion detection 중간
                    globals()['s{}_cnt'.format(i)], globals()['s{}_safe_cnt'.format(i)] = (globals()['s{}_cnt'.format(i)] + 1), (globals()['s{}_safe_cnt'.format(i)] + 1)
                    
                elif end_sensing_threshold(globals()['sensor{}'.format(i)], globals()['S{}_Threshold'.format(i)], idx) == True and globals()['s{}_cnt'.format(i)] > 0 and globals()['s{}_safe_cnt_2'.format(i)] > 0:  # Hand motion detection 끝
                    globals()['s{}_end_frame'.format(i)] = idx + 70  #0.6초 정도 뒤로
                    print(i, " 센서 넘었다가 다시 내려감", idx)
                    globals()['s{}_safe_cnt'.format(i)] = 0
                    
                elif globals()['s{}_start_frame'.format(i)] == 0 and globals()['s{}_end_frame'.format(i)] == 0: #False 일 때, end_Frame 끝나면 ㄱ
                    if idx % init_period == 0:  #매번 5초 마다 init 실행
                        globals()['offset_s{}'.format(i)] = np.mean(globals()['init_s{}'.format(i)])
                        idx_tmp = len(globals()['sensor{}'.format(i)])
                        globals()['S{}_Threshold'.format(i)] = np.mean(globals()['sensor{}'.format(i)][idx_tmp-50:idx_tmp]) + Threshold  #Th업데이트
                        globals()['init_s{}'.format(i)] = []  #offset 계산해주는 리스트 초기화
                        
                        print(i," 센서 ", idx, " 번째 idx 에서 Init, Threshold 업데이트!!! : ", np.round(globals()['S{}_Threshold'.format(i)], 3))
                
                if globals()['sensor{}'.format(i)][idx] > globals()['S{}_Threshold'.format(i)] and globals()['s{}_safe_cnt'.format(i)] > 50: # 안전장치: 계속 넘으면 다시 안내려감 = 1초 동안 넘으면 다시 내려감
                    print(i, " 센서 안전 장치! ")  #sensor 값에서 바로 추출
                    idx_tmp = len(globals()['sensor{}'.format(i)])
                    globals()['offset_s{}'.format(i)] = np.mean(globals()['sensor{}'.format(i)][idx_tmp-50:idx_tmp]) #offset 다시 설정
                    globals()['s{}_safe_cnt'.format(i)] = 0
                
                if idx == globals()['s{}_end_frame'.format(i)] and idx > 800:  #대략 맨 처음 8초 정도
                    print(i," 센서 => 리스트에 저장")
                    globals()['sensor{}_frame'.format(i)].append([globals()['s{}_start_frame'.format(i)], globals()['s{}_end_frame'.format(i)]]) 
                    globals()['s{}_end_frame'.format(i)], globals()['s{}_start_frame'.format(i)], globals()['s{}_cnt'.format(i)], globals()['s{}_safe_cnt_2'.format(i)] = 0, 0, 0, 0  #초기화
            
#-----------------------------------------------------------------------------------------------------------------------------------------           
            #맨 처음 넘은 건 바로 저장, 마지막 시점은 계속 업데이트
            if next_frame_period < idx:
                
                for k in range(1,5):
                    if not globals()['sensor{}_frame'.format(k)]:    #빈리스트 체크 = True 일 때 pass
                        pass
                    
                    else:     #빈리스트가 아니면 
                        start_frame = globals()['sensor{}_frame'.format(k)][-1][0]   #맨처음만 저장    
                        end_frame = globals()['sensor{}_frame'.format(k)][-1][1]   #계속 업데이트
                        #now1 = globals()['sensor{}_frame'.format(k)][-1][0]
                        #if now1 != 0 and start_frame == 0 and now1 > end_frame+50: 
                        #    start_frame = globals()['sensor{}_frame'.format(k)][-1][0]   #맨처음만 저장
                        #    break
                    
                        
                if  idx == end_frame and start_frame != 0 and end_frame != 0:  #start_frame + 100? end_frame?
                    Final_frame.append([start_frame, end_frame])
                    print("Final Frame ===============> ", Final_frame[-1])
                    start_frame, end_frame = 0, 0
                    for k in range(1,5):
                        globals()['sensor{}_frame'.format(k)] = []  #초기화
                    next_frame_period = idx + 400
                
            idx = idx + 1
            

In [ ]:
pass_point = 0
plt.figure(figsize=(14,7))
plt.plot(sensor1[pass_point:], label='Sensor A')
plt.plot(sensor2[pass_point:], label='Sensor B')
plt.plot(sensor3[pass_point:], label='Sensor C')
plt.plot(sensor4[pass_point:], label='Sensor D')
plt.legend(fontsize=13)
plt.ylabel('Amp (mV)')

for j in range(len(Final_frame)):
    plt.axvline(Final_frame[j][0], color='r')
    plt.axvline(Final_frame[j][1], color='b')
        
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(15,30))
j=0
letter = ['A', 'B', 'C', 'D']
pass_point = 0

for i in range(1,5):
    plt.subplot(4, 1, i)
    plt.plot(globals()['sensor{}'.format(i)][pass_point:], label='Sensor %c'%letter[i-1])
    plt.scatter(np.arange(0,len(globals()['Test_threshold{}'.format(i)])), globals()['Test_threshold{}'.format(i)], c='r')
    plt.grid(True)
    plt.legend()
    plt.ylabel('Amp (mV)')

plt.show()   

In [ ]:
plt.figure(figsize=(15,30))
j=0
letter = ['A', 'B', 'C', 'D']
pass_point = 0

for i in range(1,5):
    plt.subplot(4, 1, i)
    plt.plot(globals()['sensor{}'.format(i)][pass_point:], label='Sensor %c'%letter[i-1])
    plt.scatter(np.arange(0,len(globals()['Test_threshold{}'.format(i)])), globals()['Test_threshold{}'.format(i)], c='r')    
    plt.grid(True)
    plt.legend()
    plt.ylabel('Amp (mV)')
    
    for j in range(len(Final_frame)):
        plt.axvline(Final_frame[j][0], color='r')
        plt.axvline(Final_frame[j][1], color='b')

plt.show()   

In [ ]:
plt.figure(figsize=(15,len(Final_frame)*5))
j=0
pass_point = 0

for i in range(len(Final_frame)):
    plt.subplot(len(Final_frame), 2, i+1)
    plt.plot(sensor1[Final_frame[i][0]:Final_frame[i][1]], label='Sensor A')
    plt.plot(sensor2[Final_frame[i][0]:Final_frame[i][1]], label='Sensor B')
    plt.plot(sensor3[Final_frame[i][0]:Final_frame[i][1]], label='Sensor C')
    plt.plot(sensor4[Final_frame[i][0]:Final_frame[i][1]], label='Sensor D')
    plt.title('Frame %d'%i, fontsize=13)
    #plt.axvline(Final_frame[i][0], color='r')
    #plt.axvline(Final_frame[i][1], color='b')
    plt.grid(True)
    plt.ylabel('Amp (mV)')
    plt.legend()
    
plt.tight_layout()
plt.show()   

In [ ]:
for i in range(len(Final_frame)):
    globals()['df_frame{}'.format(i)] = []
    for j in range (1,5):
        globals()['df_frame{}'.format(i)].append(globals()['sensor{}'.format(j)][Final_frame[i][0]:Final_frame[i][1]])
        
    globals()['df_frame{}'.format(i)] = pd.DataFrame(globals()['df_frame{}'.format(i)])
    globals()['df_frame{}'.format(i)] = globals()['df_frame{}'.format(i)].transpose()
    globals()['df_frame{}'.format(i)].to_csv('C:/Users/hlee185/OneDrive - Georgia State University/2021 Fall/IoT/Dataset/test5/Down/data_%d.csv'%i)

In [ ]:
len(df_frame0), len(df_frame10), len(df_frame20), len(df_frame14)

In [ ]:
dataset = pd.read_csv(data, names=['n','a','b','c','d']).sample(n=100).sort_index().iloc[:,1:5]

In [ ]:
plt.plot(gs)

In [ ]:

2. 데이터로 저장 (csv)
3. 충분히 자른 다음에 시각화 시켜보고 데이터 추출 일단 4개 = LR, RL, CW, CCW
4. 인공지능에 학습


# FFT

In [ ]:
j=0
plt.figure(figsize=(20,20))

for i in range(1,5):
    plt.subplot(4, 2, i)
    x = np.arange(len(globals()['sensor{}'.format(i)])/2)
    fft = np.fft.fft(globals()['sensor{}'.format(i)]) / len(globals()['sensor{}'.format(i)])
    fft_magnitude = np.abs(fft)
    fft_shift = np.fft.fftshift(fft_magnitude)    
    
    plt.title('FFT sensor %d'%i, fontsize=13)
    plt.stem(x[1:], fft_shift[int(len(fft_shift)/2)+1:])
    plt.grid(True)
    
plt.tight_layout()
plt.show()